### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [3]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
def combine_data(df_o, df_inv, df_his):
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    for date_current in df_o.index:
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [6]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [7]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 이쓴
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml = '../data/data_for_ml/'
pkl_directory = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle(pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    globals()['df_{}_{}'.format(val[1], 'combined')] = combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )

In [8]:
# column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
#               'open', 'high', 'low', 'close',  'vol', 'weekday', 'value']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday', 'value' ]

In [9]:
dd = df_ktng_combined[column_selected] # company, common, df_o data combined.
dd

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc,open,high,low,close,vol,weekday,value
2022-06-20,5.957611,1.382777,0.326582,-5.192308,43.800000,0.244039,-0.226131,-5.833333,-1.414000,-8.555556,-1.363636,-inf,-0.006002,-0.003576,-0.004813,0.0,0.639612,4,2
2022-06-21,0.555802,-0.482667,-1.686069,0.642601,3.437500,-0.736189,-1.850649,-0.827586,-0.874396,-3.441176,1.625000,-0.142857,0.008454,0.001196,-0.007255,-0.010791,-0.423148,0,3
2022-06-22,0.085084,-0.499857,1.037552,0.003643,-0.730382,-0.917094,-0.022901,-1.200000,-6.048077,-1.156627,0.261905,-1.166667,-0.00479,-0.003584,0.00609,0.007273,-0.158932,1,2
2022-06-23,-0.655512,0.301026,-1.034130,-0.965884,-0.839552,3.865979,-0.593750,-0.000000,-0.643810,-0.500000,-0.622642,-13.000000,0.002407,0.003597,-0.006053,-0.01083,0.322923,2,1
2022-06-24,3.798095,-0.383436,-62.020000,54.368794,-17.558140,-0.783898,35.788462,13.500000,-6.994652,1.076923,22.750000,-0.250000,-0.004802,-0.004779,0.001218,0.004866,0.227933,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-16,0.187535,0.748920,0.450242,-3.202847,-0.566038,-0.772762,-1.042174,-inf,-0.819527,NaN,-0.846847,-1.048780,-0.004494,-0.008949,-0.007964,-0.012415,-0.186005,2,2
2023-02-17,-0.528116,-2.707317,-1.841988,-2.642703,-0.856187,0.009182,-0.173077,-1.000000,1.172131,NaN,-2.235294,0.500000,-0.012415,-0.003386,0.001147,0.0,-0.16942,3,3
2023-02-20,0.175770,-1.572514,-1.969604,-1.472218,1.418605,-0.094293,3.837209,-inf,-0.943396,-inf,-2.619048,4.666667,-0.002286,-0.010193,-0.0126,-0.004571,-0.065206,4,3
2023-02-21,-3.326858,-0.708149,-1.412313,-1.625824,-1.288462,-0.657534,-0.716346,-1.641026,-26.600000,-1.112903,-1.029412,-0.058824,-0.002291,0.016018,0.008121,0.019518,-0.068007,0,3


## 주요 지표 읽기

### Read and Combine data 

In [ ]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [10]:
# 매일 개장일자에 따른 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [11]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

##### major index 만 우선 정리하여 이용 후 하나씩 추가하여 분석하도록....

In [ ]:
major_index = ["dji.pkl", "nas.pkl", "spx.pkl", "kosdaq.pkl", "kospi.pkl"]

In [12]:
pkl_common_directory = '../data/common_pkl/'
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# for index_name in major_index:
for index_name in common_pkl:
    key_name = index_name[:-4]
#     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
#     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
                            how='left', left_index=True, right_index=True)

In [13]:
df_base.tail()

,dji_cr,dji_f_cr,dxy_cr,ixic_f_cr,bond_kor_10_cr,bond_kor_2_cr,kosdaq_cr,kospi_cr,krw_cr,ixic_cr,spx_f_cr,sox_cr,spx_cr,bond_usa_10_cr,bond_usa_2_cr,bond_usa_3m_cr,vix_cr,wti_cr
date,,,,,,,,,,,,,,,,,,
2023-02-20,NaN,-0.17%,0.00%,+0.16%,-0.22%,+1.00%,+1.71%,+0.16%,+0.05%,NaN,-0.06%,NaN,NaN,+0.91%,+1.02%,+0.54%,+6.04%,+0.82%
2023-02-21,-2.06%,-1.80%,0.32%,-2.28%,-0.30%,-0.64%,+0.57%,+0.16%,+0.75%,-2.50%,-1.74%,-3.31%,-2.00%,+2.68%,+1.39%,-0.50%,+7.72%,-1.47%
2023-02-22,-0.25%,-0.25%,0.39%,+0.01%,+1.28%,+0.83%,-1.88%,-1.68%,-0.23%,+0.13%,-0.17%,-0.48%,-0.16%,-0.69%,-0.67%,+0.25%,-2.54%,-2.90%
2023-02-23,+0.33%,+0.31%,0.03%,+0.91%,-1.59%,-1.41%,+0.61%,+0.89%,-0.36%,+0.72%,+0.49%,+3.63%,+0.53%,-1.09%,+0.18%,+0.08%,-5.16%,+2.15%
2023-02-24,NaN,-0.07%,-0.15%,-0.12%,NaN,-0.46%,+0.06%,NaN,NaN,NaN,-0.03%,NaN,NaN,NaN,NaN,+0.49%,0.00%,NaN


#### combine with 'df_xxx_combined'

In [14]:
df_sec_sel = df_sec_combined[column_selected]
df_hyunmotor_sel = df_hyunmotor_combined[column_selected]
df_naver_sel = df_naver_combined[column_selected]
df_ktng_sel = df_ktng_combined[column_selected]

In [17]:
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)

In [19]:
# 'close_cr' column 추가
pkl_d = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}_historical.pkl'.format(val[1])
    df = pd.read_pickle(pkl_d + pkl_name).set_index('date')[['close_cr']]
    df['close_cr'] = df.apply(pd.to_numeric)['close_cr'].map(lambda x : 0 if x <= 0 else ( 1 if x > 0  else  "" ))
    # 전일대비 +이면 1, -, 0 이면  0으로 변환
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df, how='left', left_index=True, right_index=True)

In [27]:
# new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity', 'open',
#   'high', 'low', 'close', 'vol', 'dji_cr', 'ixic_cr', 'spx_cr', 'kosdaq_cr', 'kospi_cr', 'weekday', 'value', 'close_cr']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust',
                'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                'corporateetc', 'foreigneretc', 'open', 'high', 'low', 'close', 'vol',
                'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr',
                'bond_kor_10_cr', 'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
                'kosdaq_cr', 'kospi_cr', 'krw_cr',
                'sox_cr', 'vix_cr', 'wti_cr', 
                'weekday', 'value', 'close_cr']

In [28]:
# column 순서 변경 : weekday, value column을 마지막으로 이동 
df_sec_sel =  df_sec_sel[new_columns]
df_hyunmotor_sel =  df_hyunmotor_sel[new_columns]
df_naver_sel =  df_naver_sel[new_columns]
df_ktng_sel =  df_ktng_sel[new_columns]

In [ ]:
'''
# drop inf, -inf : first replace inf to NaN [ bank to foreigneretc]
df_sec_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_hyunmotor_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_naver_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_ktng_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
'''

In [34]:
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_hyunmotor_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_naver_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_ktng_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)

In [36]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
df_sec_sel.dropna(inplace=True)
df_hyunmotor_sel.dropna(inplace=True)
df_naver_sel.dropna(inplace=True)
df_ktng_sel.dropna(inplace=True)

In [37]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)

df_sec_sel = string_to_num(df_sec_sel)
df_hyunmotor_sel = string_to_num(df_hyunmotor_sel)
df_naver_sel = string_to_num(df_naver_sel)
df_ktng_sel = string_to_num(df_ktng_sel)

In [38]:
# pickle로 데이터 저장
directory_for_ml = '../data/data_for_ml/'
df_sec_sel.to_pickle(directory_for_ml + 'df_sec_sel.pkl')
df_hyunmotor_sel.to_pickle(directory_for_ml + 'df_hyunmotor_sel.pkl')
df_naver_sel.to_pickle(directory_for_ml + 'df_naver_sel.pkl')
df_ktng_sel.to_pickle(directory_for_ml + 'df_ktng_sel.pkl')

df_sec_sel.to_csv(directory_for_ml + 'df_sec_sel.csv')
df_hyunmotor_sel.to_csv(directory_for_ml + 'df_hyunmotor_sel.csv')
df_naver_sel.to_csv(directory_for_ml + 'df_naver_sel.csv')
df_ktng_sel.to_csv(directory_for_ml + 'df_ktng_sel.csv')

### feature 이름 설명
#### feature에 사용된 column name 설명

In [23]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [24]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [26]:
# financial이 무엇인지 알기 위한 명령
c_name = 'dxy_cr'
standforK(c_name), standforE(c_name), standforexp(c_name)

('미국달러지수 변동률', 'Dollar_index_change_rate', nan)

In [ ]:
c_name = 'cpi'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'sox'
standforK(c_name), standforE(c_name), standforexp(c_name)

### 설명에서 feature name 찾기

In [ ]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [ ]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

In [ ]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

#### pd.merge 형식 참조
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)

In [ ]:
bokd = '../data/common_pkl/bok_rate.pkl'
cpid = '../data/common_pkl/cpi.pkl'
djid = '../data/common_pkl/dji.pkl'
bok = pd.read_pickle(bokd)
cpi = pd.read_pickle(cpid)
dji = pd.read_pickle(djid)
boks = pd.read_pickle(bokd).set_index('date')
cpis = pd.read_pickle(cpid).set_index('date')
djis = pd.read_pickle(djid).set_index('date')

In [ ]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [ ]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [ ]:
df_base = df_base.merge(cpis, how='left', left_index=True, right_index=True)

In [ ]:
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)